In [71]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
plt.rcParams["figure.figsize"] = "15, 8"

import pandas as pd
import numpy as np

In [7]:
#http://stackoverflow.com/questions/8897593/similarity-between-two-text-documents
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [8]:
lines = []
with open('sotu.txt') as f:
    for line in f:
        lines.append(line.strip())

lines = [line for line in lines if line != '\n' and line != '']
lines.append('')

stars = [i for i, x in enumerate(lines) if x == '***']
# last line of file
stars.append(len(lines) - 1)

def president(i):
    return ' '.join(lines[stars[i] + 2 : stars[i] + 3]).strip()

def date(i):
    return ' '.join(lines[stars[i] + 3 : stars[i] + 4]).strip()

def speech(i):
    return ' '.join(lines[stars[i] + 4 : stars[i + 1]]).strip()

In [43]:
def rolling_similarity(end_time):
    similarities = []
    for i in range(end_time):
        similarities.append(cosine_sim(speech(i), speech(i + 1)))
    return similarities

In [44]:
diff_one_series = rolling_similarity(223)

In [106]:
years = []
for i in range(224):
    years.append(date(i)[-4:])

In [115]:
pd.DataFrame({"1790":[.5, .7, .8]})

,1790
0,0.5
1,0.7
2,0.8


In [150]:
df = pd.DataFrame()
dfs = []
for i, year in enumerate(years[:10]):
    print(year)
    first = speech(i)
    temp_dist = []
    temp_df = pd.DataFrame()
    for j, _ in enumerate(years[:10]):
        if j < i:
            temp_dist.append('-')
        else:
            temp_dist.append(cosine_sim(first, speech(j)))
    temp_df[year] = temp_dist
    dfs.append(temp_df)
df = pd.concat(dfs, axis=1)

1790
1790
1791
1792
1793
1794
1795
1796
1797
1798


In [153]:
df

,1790,1790,1791,1792,1793,1794,1795,1796,1797,1798
0,1.000000,-,-,-,-,-,-,-,-,-
1,0.438101,1,-,-,-,-,-,-,-,-
2,0.497558,0.470993,1,-,-,-,-,-,-,-
3,0.428291,0.471359,0.548411,1,-,-,-,-,-,-
4,0.404396,0.420486,0.578943,0.475358,1,-,-,-,-,-
5,0.378920,0.402047,0.514577,0.45431,0.589911,1,-,-,-,-
6,0.413115,0.453712,0.515655,0.468445,0.472821,0.460478,1,-,-,-
7,0.433454,0.461258,0.52338,0.439472,0.567868,0.497958,0.503804,1,-,-
8,0.358947,0.388894,0.493629,0.411983,0.557373,0.466348,0.441189,0.566396,1,-
9,0.392483,0.43603,0.476434,0.415894,0.530554,0.471653,0.439009,0.597472,0.58356,1


In [96]:
np.sort(diff_one_series)[:-1][:10]

array([ 0.26909885,  0.34253401,  0.41171552,  0.41380018,  0.43810108,
        0.45164537,  0.45481508,  0.45583735,  0.46047781,  0.46178195])